# Lab02: Frequent itemset mining

- Student ID: 20120128
- Student name: Nguyễn Thị Cẩm Lai

**How to do your homework**


You will work directly on this notebook; the word `TODO` indicate the parts you need to do.

You can discuss ideas with classmates as well as finding information from the internet, book, etc...; but *this homework must be your*.

**How to submit your homework**

Before submitting, rerun the notebook (`Kernel` ->` Restart & Run All`).

Then create a folder named `ID` (for example, if your ID is 1234567, then name the folder `1234567`) Copy file notebook to this folder, compress and submit it on moodle.

**Contents:**

- Frequent itemset mining.

# 1. Preliminaries
## This is how it all started ...
- Rakesh Agrawal, Tomasz Imielinski, Arun N. Swami: Mining Association Rules between Sets of Items in Large Databases. SIGMOD Conference 1993: 207-216
- Rakesh Agrawal, Ramakrishnan Srikant: Fast Algorithms for Mining Association Rules in Large Databases. VLDB 1994: 487-499

**These two papers are credited with the birth of Data Mining**
## Frequent itemset mining (FIM)

Find combinations of items (itemsets) that occur frequently.
## Applications
- Items = products, transactions = sets of products someone bought in one trip to the store.
$\Rightarrow$ items people frequently buy together.
    + Example: if people usually buy bread and coffee together, we run a sale of bread to attract people attention and raise price of coffee.
- Items = webpages, transactions = words. Unusual words appearing together in a large number of documents, e.g., “Brad” and “Angelina,” may indicate an interesting relationship.
- Transactions = Sentences, Items = Documents containing those sentences. Items that appear together too often could represent plagiarism.

## Transactional Database
A transactional database $D$ consists of $N$ transactions: $D=\left\{T_1,T_2,...,T_N\right\}$. A transaction $T_n \in D (1 \le n \le N)$ contains one or more items and that $I= \left\{ i_1,i_2,…,i_M \right\}$ is the set of distinct items in $D$, $T_n \subset I$. Commonly, a transactional database is represented by a flat file instead of a database system: items are non-negative integers, each row represents a transaction, items in a transaction separated by space.

Example: 

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

30 31 32 

33 34 35 

36 37 38 39 40 41 42 43 44 45 46 

38 39 47 48 

38 39 48 49 50 51 52 53 54 55 56 57 58 

32 41 59 60 61 62 

3 39 48 

63 64 65 66 67 68 



# Definition

- Itemset: A collection of one or more items.
    + Example: {1 4 5}
- **k-itemset**: An itemset that contains k items.
- Support: Frequency of occurrence of an itemset.
    + Example: From the example above, item 3 appear in 2 transactions so its support is 2.
- Frequent itemset: An itemset whose support is greater than or equal to a `minsup` threshold

# The Apriori Principle
- If an itemset is frequent, then all of its subsets must also be frequent.
- If an itemset is not frequent, then all of its supersets cannot be frequent.
- The support of an itemset never exceeds the support of its subsets.
$$ \forall{X,Y}: (X \subseteq Y) \Rightarrow s(X)\ge s(Y)$$


# 2. Implementation


## The Apriori algorithm
Suppose:

$C_k$ candidate itemsets of size k.

$L_k$ frequent itemsets of size k.

The level-wise approach of Apriori algorithm can be descibed as follow:
1. k=1, $C_k$ = all items.
2. While $C_k$ not empty:
    3. Scan the database to find which itemsets in $C_k$ are frequent and put them into $L_k$.
    4. Use $L_k$ to generate a collection of candidate itemsets $C_{k+1}$ of size k+1.
    5. k=k+1.

### Import library

In [6]:
from collections import defaultdict
import csv

### Read data
First we have to read data from database

In [7]:

def readData(path):
    """
    Parameters
    --------------------------
        path: path of database D.
         
    --------------------------
    Returns
        data: a dictionary for representing database D
                 - keys: transaction tids
                 - values: itemsets.
        s: support of distict items in D.
    """
    data={}
    s=defaultdict(lambda: 0) # Initialize a dictionary for storing support of items in I.  
    with open(path,'rt') as f:
        tid=1;
        for line in f:
            itemset=set(map(int,line.split())) # a python set is a native way for storing an itemset.
            for item in itemset:  
                s[item]+=1     #Why don't we compute support of items while reading data?
            data[tid]= itemset
            tid+=1
    
    return data, s

### Tree Projection

**I gave you pseudo code of Apriori algorithm above but we implement Tree Projection. Tell me the differences of two algorithms.**


**TODO:**

In [8]:
def joinset(a, b):
    '''
    Parameters
    -------------------
        2 itemsets a and b (of course they are at same branch in search space)

    -------------------
    return
        ret: itemset generated by joining a and b
    '''
    # TODO (hint: this function will be called in generateSearchSpace method.):
    ret=list(set(a) | set(b))
    return ret

class TP:
    def __init__(self, data=None, s=None, minSup=None):
        self.data = data
        self.s = {}

        for key, support in sorted(s.items(), key=lambda item: item[1]):
            self.s[key] = support
        # TODO: why should we do this, answer it at the markdown below?

        self.minSup = minSup
        self.L = {}  # Store frequent itemsets mined from database
        self.runAlgorithm()

    def initialize(self):
        """
        Initialize search space at first step
        --------------------------------------
        We represent our search space in a tree structure
        """
        tree = {}

        search_space = {}
        for item, support in self.s.items():
            search_space[item] = {}

            search_space[item]['itemset'] = [item]
            ''' 
            python set does not remain elements order
            so we use a list to extend it easily when create new itemset 
            but why we store itemset in data by a python set???? '''
            # TODO: study about python set and its advantages,
            # answer at the markdown below.

            search_space[item]['pruned'] = False
            # TODO:
            # After finish implementing the algorithm tell me why should you use this
            # instead of delete item directly from search_space and tree.

            search_space[item]['support'] = support

            tree[item] = {}
            '''
            Why should i store an additional tree (here it called tree)? 
            Answer: This really help in next steps.

            Remember that there is always a big gap from theory to practicality
            and implementing this algorithm in python is not as simple as you think.
            '''

        return tree, search_space

    def computeItemsetSupport(self, itemset):

        '''Return support of itemset'''
        # TODO (hint: this is why i use python set in data)
        support=0
        for i in self.data.keys():
            if len(set(itemset)) == len(set(itemset).intersection(self.data[i])):
                support+=1
        return support
    
    def get_sub_tree(self, k, tree, search_space, itter_node):
        if k == 0:
            return search_space[itter_node]['support']
        subtree = search_space[itter_node]
        for node in subtree.keys():
            k-=1
            self.get_sub_tree(k,tree,search_space,node)

    def prune(self, k, tree, search_space):

        '''
        In this method we will find out which itemset in current search space is frequent
        itemset then add it to L[k]. In addition, we prune those are not frequent itemsets.
        '''
        if self.L.get(k) is None: self.L[k] = []
        # TODO
        for key in search_space.keys():
            if search_space[key]['support']<self.minSup:#isn't frequent itemsets
                tree.pop(key)
                search_space[key]['pruned']=True
            else:
                search_space[key]['pruned']=False
                self.L[k].append(search_space[key]['itemset'])

    def generateSearchSpace(self, k, tree, search_space):
        '''
        Generate search space for exploring k+1 itemset. (Recursive function)
        '''
        items = list(tree.keys())
        ''' print search_space.keys() you will understand  
         why we need an additional tree, '''
        
        l = len(items)
        self.prune(k, tree, search_space)
        if l == 0: return  # Stop condition
        for i in range(l - 1):
            sub_search_space = {}
            sub_tree = {}
            a = items[i]
            if search_space[a]['pruned']: continue

            for j in range(i + 1, l):
                b = items[j]
                search_space[a][b] = {}
                tree[a][b] = {}
                # You really need to understand what am i doing here before doing work below.
                # (Hint: draw tree and search space to draft).

                # TODO:
                # First create newset using join set
                newset=joinset(search_space[a]['itemset'],search_space[b]['itemset'])
                # Second add newset to search_space
                search_space[a][b]={'itemset':list(newset),'pruned':False,'support':self.computeItemsetSupport(newset)}
                sub_search_space[b]=search_space[a][b]
                sub_tree[b]={}
            #  Generate search_space for k+1-itemset
            self.generateSearchSpace(k + 1, sub_tree, sub_search_space)

    def runAlgorithm(self):
        tree, search_space = self.initialize()  # generate search space for 1-itemset
        self.generateSearchSpace(1, tree, search_space)

    def miningResults(self):
        return self.L

Ok, let's test on a typical dataset `chess`.

In [9]:
data, s= readData('chess.txt')

In [10]:
a=TP(data=data,s=s, minSup=3000)
print(a.miningResults())

{1: [[48], [56], [66], [34], [62], [7], [36], [60], [40], [29], [52], [58]], 2: [[48, 52], [48, 58], [56, 29], [56, 52], [56, 58], [66, 60], [66, 29], [66, 52], [66, 58], [40, 34], [34, 29], [34, 52], [34, 58], [60, 62], [40, 62], [29, 62], [52, 62], [58, 62], [60, 7], [40, 7], [29, 7], [52, 7], [58, 7], [36, 60], [40, 36], [36, 29], [36, 52], [58, 36], [40, 60], [60, 29], [60, 52], [58, 60], [40, 29], [40, 52], [40, 58], [52, 29], [58, 29], [58, 52]], 3: [[48, 58, 52], [56, 52, 29], [56, 58, 29], [56, 58, 52], [66, 60, 29], [66, 60, 52], [66, 60, 58], [66, 52, 29], [66, 58, 29], [66, 52, 58], [40, 34, 29], [40, 34, 52], [40, 34, 58], [34, 52, 29], [34, 58, 29], [34, 52, 58], [60, 29, 62], [60, 62, 52], [58, 60, 62], [40, 29, 62], [40, 52, 62], [40, 58, 62], [52, 29, 62], [58, 29, 62], [58, 52, 62], [40, 60, 7], [60, 29, 7], [60, 52, 7], [58, 60, 7], [40, 29, 7], [40, 52, 7], [40, 58, 7], [52, 29, 7], [58, 29, 7], [58, 52, 7], [40, 36, 60], [36, 29, 60], [36, 60, 52], [58, 36, 60], [40

### Answer questions here:
**1. Why don't we compute support of items while reading data?**

We should not calculate support for items when reading data because support for itemset will be repeated many times later for other L-itemsets (with L>1). So in `class TP` there is a function to calculate its own support.

**2. Why should we do sort**

Sorting the items in self.s (`s.items()`) in ascending order of support will help determine if an itemset is not frequent, it will have the smallest number of items.

**3. Study about python set and its advantages ?**

- In Python, set is a data type where a set of elements is unordered and duplicate elements are not allowed. Set in Python is different from List or Tuple because both of these data types allow to store duplicate elements and have a clear order.

Set in Python has various methods that allow you to perform operations with set. Some of the most popular methods include:

     - add(): Add an element to set
     - update(): Add more elements to the set from an iterable object, such as a list or a tuple
     - remove(): Removes an element from the set. If the element does not exist, the method will raise a KeyError.
     - discard(): Removes an element from the set. If the element does not exist, the method will not raise an error.
     - union(): Returns a new set containing all the elements of both or more sets.
     intersection(): Returns a new set containing the elements common to two or more sets.

The elements of a set can be of any data type, including numeric, string, tuple, etc.

- Set is unordered: performing some calculations is easy because we do not need to care about the order of the elements in the set.
     - For example, searching for an element or checking if an element exists in a set.
- Set is immutable object: However, the elements in the set must be immutable.

Set provides fast access to elements: Since set is implemented using a hash table, it is quick to access the elements of the set.

**4. After finish implementing the algorithm tell me why should you use this? Instead of delete item directly from search_space and tree.**

Because the algorithm uses recursion (in the generateSearchSpace function) to find itemsets that are frequent sets (starting from the space k=1, then increasing to k=2,3,4,... after each time). recursive). So to determine if itemset is common or not without affecting the result when calling the function recursively, we use the variable `pruned (=true/false)` instead of deleting them directly in search_space and tree.

**5. Apriori algorithm and Tree Projection, tell me the differences of two algorithms.**

Apriori and Tree Projection algorithms are the two most popular algorithms in association rule mining to find common itemsets in a data set. Here are some of the differences between these two algorithms:

1. Storage space: Tree Projection algorithm is more efficient for processing larger data. Because of the Tree Projection data structure it uses, it needs a smaller storage space than Apriori. This makes Tree Projection work faster, especially with large data sets.

2. Approach: Apriori uses combinatorial system approach to determine association rules, while Tree Projection uses tree query approach to data mining.

3. Coverage: Apriori often finds itemsets with higher overall coverage than Tree Projection, especially when the data set is not too large. This can be explained by the way Apriori finds all the itemsets in each supporting set, until there are no more matching itemsets.

4. Mining: Apriori searches for all the frequent items in the data set, then uses them to identify combinations. Meanwhile, Tree Projection uses tree structure to search for common data patterns and generate association rules.


In summary, both Apriori and Tree Projection are powerful algorithms in association rule mining to find common itemsets in a data set. Apriori is simpler and easier to understand, but its limitation is that it is not efficient when dealing with large data sets. Meanwhile, Tree Projection is faster for large data sets and allows saving on storage costs, but the returned results may not have as high overall coverage as Apriori offers.

**6. Why should we do this** `(self.s[key] = support)`

- To get the support value corresponding to each itemset in the dataset `chess.txt`.

# 3. Churn analysis

In this section, you will use frequent itemset mining technique to analyze `churn` dataset (for any purposes). 

*Remember this dataset is not represented as a transactional database, first thing that you have to do is transforming it into a flat file.  

### 3.0 About dataset

**Attribute meaning**

`State`: the US state in which the customer resides, indicated by a two-letter abbreviation

`Account Length`: the number of days that this account has been active

`Area Code`: the three-digit area code of the corresponding customer’s phone number

`Phone`: the remaining seven-digit phone number

`Int’l Plan`: whether the customer has an international calling plan: yes/no

`VMail Plan`: whether the customer has a voice mail feature: yes/no

`VMail Message`: presumably the average number of voice mail messages per month

`Day Mins`: the total number of calling minutes used during the day

`Day Calls`: the total number of calls placed during the day

`Day Charge`: the billed cost of daytime calls

`Eve Mins`: the total number of calling minutes used during the evening

`Eve Calls`: the total number of calls placed during the evening

`Eve Charge`: the billed cost of evening time calls

`Night Mins`: the total number of calling minutes used during the night

`Night Calls`: the total number of calls placed during the night

`Night Charge`: the billed cost of nighttime calls

`Intl Mins`: the total number of international minutes

`Intl Calls`: the total number of international calls

`Intl Charge`: the billed cost for international calls

`CustServ Calls`: the number of calls placed to Customer Service

`Churn?`: whether the customer left the service: true/false

### 3.1 Transforming `churn.txt` to transactional database

In [25]:
# Define read_large_file()
def readLargeFile(filename):
    f = open(filename, 'r')
    for line in f:
        data = line.split(',')
        data[0] = "State: " + data[0]
        data[1] = "Account Length: " + data[1]
        data[2] = "Area Code: " + data[2]
        data[3] = "Phone: " + data[3]
        data[4] = "Int'l Plan: " + data[4]
        data[5] = "VMail Plan: " + data[5]
        data[6] = "VMail Message: " + data[6]
        data[7] = "Day Mins: " + data[7]
        data[8] = "Day Calls: " + data[8]
        data[9] = "Day Charge: " + data[9]
        data[10] = "Eve Mins: " + data[10]
        data[11] = "Eve Calls: " + data[11]
        data[12] = "Eve Charge: " + data[12]
        data[13] = "Night Mins: " + data[13]
        data[14] = "Night Calls: " + data[14]
        data[15] = "Night Charge: " + data[15]
        data[16] = "Intl Mins: " + data[16]
        data[17] = "Intl Calls: " + data[17]
        data[18] = "Intl Charge: " + data[18]
        data[19] = "CustServ Calls: " + data[19]
        data[20] = "Churn?: " + data[20][:-1]
        if not data:
            break
        yield data

In [12]:
data=readLargeFile("churn.txt")

**Lưu ở định dạng .csv `churn.csv`**

In [13]:
filename = "churn.csv"
    
# writing to csv file 
with open(filename, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    for i in data:
        csvwriter.writerow(i) 

### 3.2 Churn analysis with Apriori

In [14]:
def selfjoin(a, b):
    ret = list(set(a) | set(b))    
    return ret

In [15]:
#Lấy danh sách các giao dịch
list_transactions = []
list_row = csv.reader(open(filename,'r', encoding='utf-8'))
for row in list_row:
    list_transactions.append(sorted(row))
delete_fist_row=list_transactions.pop(0) #dòng đầu vì nó là tên các thuộc tính

In [16]:
print('Number of transaction in churn: ', len(list_transactions))

Number of transaction in churn:  3333


In [17]:
#danh sách các item và giá trị mà nó có
list_items = []
for i in list_transactions:
    for j in i:
        if j not in list_items:
            list_items.append(j)
list_items = sorted(list_items)

In [18]:
#Tìm tập hợp các transaction chứa X
def findItemsetCover(X, list_transactions):
    list_transaction_cover_X = []
    for i in range(len(list_transactions)):
        if set(X).issubset(list_transactions[i]):
            list_transaction_cover_X.append(i)
    return list_transaction_cover_X

In [19]:
#Tính số lượng các transaction chứa X (độ phổ biến theo số lượng)
def calAbsoluteSupport(X, list_transactions):
    abs_support = len(findItemsetCover(X, list_transactions))
    return abs_support

In [20]:
#Kiểm tra xem liệu itemset X có là tập phổ biến
def isFrequentItemset(X, list_transactions, min_sup):
    support_rate=round(calAbsoluteSupport(X, list_transactions)/len(list_transactions),2)
    if support_rate >= min_sup:
        return True

In [21]:
#bước cắt tỉa bởi thuật toán Apriori
def pruneByApriori(list_candidate, list_previous_frequent_itemset):
    new_list_candidate = list_candidate.copy()
    for i in list_candidate:
        for j in i:
            candidate = i.copy()
            candidate.remove(j)
            if candidate not in list_previous_frequent_itemset:
                new_list_candidate.remove(i)
                break
    sorted(new_list_candidate)
    return new_list_candidate

In [22]:
#Khai thác tập phổ biến bằng thuật toán Apriori
def findFIMApriori(list_transactions, list_items, min_sup):#FIM: Frequent itemset mining
    list_frequent_itemset = []
    list_previous_frequent_itemset = []
    
    for item in list_items:
        if isFrequentItemset([item],list_transactions,min_sup):
            list_frequent_itemset.append([item])
            list_previous_frequent_itemset.append([item])
            
    while True:
        list_candidate = []
        for i in range(len(list_previous_frequent_itemset)):
            for j in range(i+1,len(list_previous_frequent_itemset)):
                newjoin = selfjoin(list_previous_frequent_itemset[i],list_previous_frequent_itemset[j])
                if newjoin != []:
                    list_candidate.append(newjoin)
                    
        pruned = pruneByApriori(list_candidate,list_previous_frequent_itemset)
        list_previous_frequent_itemset = []
        for i in pruned:
            if isFrequentItemset(i,list_transactions,min_sup): 
                list_previous_frequent_itemset.append(i)  
                
        if list_previous_frequent_itemset == []:
            break
        for i in list_previous_frequent_itemset:
            list_frequent_itemset.append(i)
    return list_frequent_itemset

### 3.3 Test on our dataset

**min_support = 0.5**

In [23]:
# Test on our dataset
min_support = 0.5
list_fim = findFIMApriori(list_transactions, list_items, min_support)
print('List Frequent itemset mining in churn dataste with min_support = :', min_support)
for fim in list_fim:
    print(fim)

List Frequent itemset mining in churn dataste with min_support = : 0.5
['Area Code: 415']
['Churn?: False.']
["Int'l Plan: no"]
['VMail Message: 0']
['VMail Plan: no']
["Int'l Plan: no", 'Churn?: False.']
['VMail Message: 0', 'Churn?: False.']
['VMail Plan: no', 'Churn?: False.']
["Int'l Plan: no", 'VMail Message: 0']
['VMail Plan: no', "Int'l Plan: no"]
['VMail Plan: no', 'VMail Message: 0']
["Int'l Plan: no", 'VMail Message: 0', 'Churn?: False.']
['VMail Plan: no', "Int'l Plan: no", 'Churn?: False.']
["Int'l Plan: no", 'VMail Message: 0', 'Churn?: False.']
['VMail Plan: no', "Int'l Plan: no", 'Churn?: False.']
['VMail Plan: no', 'VMail Message: 0', 'Churn?: False.']
["Int'l Plan: no", 'VMail Message: 0', 'Churn?: False.']
['VMail Plan: no', 'VMail Message: 0', 'Churn?: False.']
['VMail Plan: no', "Int'l Plan: no", 'Churn?: False.']
['VMail Plan: no', 'VMail Message: 0', 'Churn?: False.']
['VMail Plan: no', "Int'l Plan: no", 'VMail Message: 0']
['VMail Plan: no', "Int'l Plan: no", 'VM

With min_support is 0.5
- No Frequent itemset churn = True.
- There is no factor that causes customers to abandon the service (churn=True)

**min_support = 0.2**

In [24]:
# Test on our dataset
min_support = 0.2
list_fim = findFIMApriori(list_transactions, list_items, min_support)
print('List Frequent itemset mining in churn dataste with min_support = :', min_support)
for fim in list_fim:
    print(fim)

List Frequent itemset mining in churn dataste with min_support = : 0.2
['Area Code: 408']
['Area Code: 415']
['Area Code: 510']
['Churn?: False.']
['CustServ Calls: 0']
['CustServ Calls: 1']
['CustServ Calls: 2']
["Int'l Plan: no"]
['Intl Calls: 3']
['VMail Message: 0']
['VMail Plan: no']
['VMail Plan: yes']
['Area Code: 408', 'Churn?: False.']
["Int'l Plan: no", 'Area Code: 408']
['Area Code: 415', 'Churn?: False.']
["Int'l Plan: no", 'Area Code: 415']
['Area Code: 415', 'VMail Message: 0']
['VMail Plan: no', 'Area Code: 415']
['Area Code: 510', 'Churn?: False.']
["Int'l Plan: no", 'Area Code: 510']
['CustServ Calls: 1', 'Churn?: False.']
['CustServ Calls: 2', 'Churn?: False.']
["Int'l Plan: no", 'Churn?: False.']
['VMail Message: 0', 'Churn?: False.']
['VMail Plan: no', 'Churn?: False.']
['VMail Plan: yes', 'Churn?: False.']
["Int'l Plan: no", 'CustServ Calls: 1']
['VMail Message: 0', 'CustServ Calls: 1']
['VMail Plan: no', 'CustServ Calls: 1']
["Int'l Plan: no", 'CustServ Calls: 2']

- Reduce min_support=0.2, also does not appear churn=True.

- It can be seen that even with a low min_support, there are no customers who refuse the service, so it can be assessed that this service is quite popular with customers.

# 4 References

[1] https://deepnote.com/@portfolio-evelin/Customer-Churn-Prediction-3a21cad9-2991-4412-8100-5091e8531328

[2] Chat GPT

Feel free to send questions to my email address: nnduc@fit.hcmus.edu.vn
